In [1]:
import sys
sys.path.append("/pod/2/ke-lab/LUOZ/Singularity/iM6A")

In [2]:
from keras.models import load_model
from pkg_resources import resource_filename
import numpy as np
import pandas as pd
from Bio.Seq import Seq
from Bio.SeqUtils import nt_search
import keras.backend as kb

Using TensorFlow backend.


In [3]:
def one_hot_encode(seq):

    map = np.asarray([[0, 0, 0, 0],
                      [1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])

    seq = seq.upper().replace('A', '\x01').replace('C', '\x02')
    seq = seq.replace('G', '\x03').replace('T', '\x04').replace('N', '\x00')

    return map[np.fromstring(seq, np.int8) % 5]

In [4]:
def categorical_crossentropy_2d(y_true, y_pred):
    # Standard categorical cross entropy for sequence outputs

    return - kb.mean(y_true[:, :, 0]*kb.log(y_pred[:, :, 0]+1e-10)
                   + y_true[:, :, 1]*kb.log(y_pred[:, :, 1]+1e-10))

In [5]:
context = 10000

In [6]:
paths = ('Test/mouseRAC10000_c{}.h5'.format(x) for x in range(1, 6))

In [7]:
models = [load_model(resource_filename('m6AAI', x), custom_objects={'categorical_crossentropy_2d': categorical_crossentropy_2d}) for x in paths]

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


### Read Fasta

In [9]:
Fasta = pd.read_csv("Fasta_Stop_Codon.csv")

In [10]:
Group = pd.read_csv("Group.csv")

In [11]:
Group = Group[["name","chrom","strand","LastExonStart","LastExonEnd","Group"]]

In [12]:
Fasta = pd.merge(Fasta, Group, how="inner", on=["name","chrom","strand","Group"])

In [13]:
Fasta

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,StopCodon,Group,Sequence,Length,LastExonStart,LastExonEnd
0,Xkr4,chr1,-,3214481,3671498,3216021,3671348,TAA,2,AAAGTTTGACCGGCACTTTTAATGCAACAAAAAGCACCATGACCAA...,457017,3214481,3216968
1,Rp1,chr1,-,4290845,4409241,4292980,4409187,TAA,2,gactgtgggtttgcaatatattgtgtttattatggagaggtatatc...,118396,4290845,4293012
2,Sox17,chr1,-,4491389,4497354,4491715,4495155,TGA,2,AAGATTCCAGGATTTATTTAAGATTGAGAAAACACGCATGACAAAA...,5965,4491389,4492668
3,Mrpl15,chr1,-,4773210,4785739,4776463,4785677,TGA,2,AGATTTAGGAATTCTGTTTATTGAACAAACTGCAGAAGGGAAGACT...,12529,4773210,4776801
4,Lypla1,chr1,+,4807822,4846739,4807913,4845016,TGA,2,gagcgcgcgcccggaggggcgggcTGGGACTTTCGGCTGCCGGGAG...,38917,4844962,4846739
5,Gm37988,chr1,+,4807891,4886770,4807913,4832316,TGA,3,CGCACTGTCCGCCAGCCGGTGGATGTGCGGCAACAACATGTCCGCT...,78879,4886743,4886770
6,Tcea1,chr1,+,4857813,4897905,4857913,4896364,TGA,2,GGAAGGCCTAGGCCAGCGGCTTCGCGGCTTGTCCAACGTCCGCGCA...,40092,4896355,4897905
7,Rgs20,chr1,-,4909575,5070285,4910473,5070177,TAG,2,CATTAAAAATTAAAATATGTTTATTTGAGAAATTGTGAACATGACA...,160710,4909575,4910662
8,Atp6v1h,chr1,+,5083079,5162529,5084450,5162165,TGA,2,AGCTGCGGTCACGTGCGGCTGCTCACGTGCCAGACGCTGCTGTCAC...,79450,5162104,5162529
9,Oprk1,chr1,+,5588492,5606131,5589048,5602784,TGA,2,GCGCACCTTGCTGATCCCAAACAGGCAGAGCTTCTTCCAGTCTTGG...,17639,5602251,5606131


In [14]:
Fasta["GeneLength"] = Fasta["txEnd"] - Fasta["txStart"]
Fasta["LastExonLength"] = Fasta["LastExonEnd"] - Fasta["LastExonStart"]
Fasta["Pre"] = Fasta["GeneLength"] - Fasta["LastExonLength"]

In [15]:
Positive = Fasta[Fasta["strand"]=="+"]
Positive = Positive.reset_index(drop = True)
Negative = Fasta[Fasta["strand"]=="-"]
Negative = Negative.reset_index(drop = True)

In [16]:
Positive["CodingLength"] = Positive["cdsEnd"] - Positive["LastExonStart"]
Negative["CodingLength"] = Negative["LastExonEnd"] - Negative["cdsStart"]

In [17]:
Fasta = Positive.append(Negative)
Fasta = Fasta.reset_index(drop = True)
Fasta["NoncodingLength"] = Fasta["LastExonLength"] - Fasta["CodingLength"]

In [18]:
Positive = Fasta[Fasta["strand"]=="+"]
Positive = Positive.reset_index(drop = True)
Negative = Fasta[Fasta["strand"]=="-"]
Negative = Negative.reset_index(drop = True)

In [19]:
Sequence = Negative["Sequence"].tolist()
New = []
for i in range(len(Sequence)):
    input_sequence = Sequence[i][::-1]
    input_sequence = Seq(input_sequence).complement()
    input_sequence = str(input_sequence)
    New.append(input_sequence)

New = np.array(New)
New = pd.DataFrame({"Sequence":New})
Negative["Sequence"] = 0
Negative["Sequence"] = New["Sequence"]

In [20]:
Fasta = Positive.append(Negative)
Fasta = Fasta.reset_index(drop = True)

In [21]:
Fasta

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,StopCodon,Group,Sequence,Length,LastExonStart,LastExonEnd,GeneLength,LastExonLength,Pre,CodingLength,NoncodingLength
0,Lypla1,chr1,+,4807822,4846739,4807913,4845016,TGA,2,gagcgcgcgcccggaggggcgggcTGGGACTTTCGGCTGCCGGGAG...,38917,4844962,4846739,38917,1777,37140,54,1723
1,Gm37988,chr1,+,4807891,4886770,4807913,4832316,TGA,3,CGCACTGTCCGCCAGCCGGTGGATGTGCGGCAACAACATGTCCGCT...,78879,4886743,4886770,78879,27,78852,-54427,54454
2,Tcea1,chr1,+,4857813,4897905,4857913,4896364,TGA,2,GGAAGGCCTAGGCCAGCGGCTTCGCGGCTTGTCCAACGTCCGCGCA...,40092,4896355,4897905,40092,1550,38542,9,1541
3,Atp6v1h,chr1,+,5083079,5162529,5084450,5162165,TGA,2,AGCTGCGGTCACGTGCGGCTGCTCACGTGCCAGACGCTGCTGTCAC...,79450,5162104,5162529,79450,425,79025,61,364
4,Oprk1,chr1,+,5588492,5606131,5589048,5602784,TGA,2,GCGCACCTTGCTGATCCCAAACAGGCAGAGCTTCTTCCAGTCTTGG...,17639,5602251,5606131,17639,3880,13759,533,3347
5,Rb1cc1,chr1,+,6214644,6276648,6230002,6274275,TAG,2,GCTCTGCGGCGGAAAGGCCGAGTCGACAATAACAAACCCCACGGCG...,62004,6274197,6276648,62004,2451,59553,78,2373
6,Fam150a,chr1,+,6359217,6394731,6359330,6391104,TAG,3,CGCCCCGCCTCCGGTCCCCGCCCAGCGCCCTAGAGCGTTTAGACCT...,35514,6394269,6394731,35514,462,35052,-3165,3627
7,St18,chr1,+,6487230,6860934,6768897,6858600,TAG,2,GAAAAGAGTTTTGAAATGAGTTCTGAATAGACAGCATTTTCTTTGT...,373704,6858459,6860934,373704,2475,371229,141,2334
8,Pcmtd1,chr1,+,7088919,7173628,7120308,7169882,TAA,2,GGCATCTTTCTTCGGAGGGCGCCCCGCGCACGCGCAGCTCCCAGGA...,84709,7169514,7173628,84709,4114,80595,368,3746
9,Rrs1,chr1,+,9545407,9547455,9545523,9546621,TAG,1,TCTTCCGGATTACGGGTCCGGACGTCCGCACGTGGTTGCCGGTTTA...,2048,9545407,9547455,2048,2048,0,1214,834


In [22]:
Fasta["PreSequence"] = 0
Fasta["LastExonSequence"] = 0

In [23]:
for i in range(len(Fasta)):
    Fasta.loc[i,("PreSequence")] = Fasta.loc[i,("Sequence")][:(Fasta.loc[i,("Pre")])]
    Fasta.loc[i,("LastExonSequence")] = Fasta.loc[i,("Sequence")][(Fasta.loc[i,("Pre")]):]

In [24]:
Fasta

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,StopCodon,Group,Sequence,Length,LastExonStart,LastExonEnd,GeneLength,LastExonLength,Pre,CodingLength,NoncodingLength,PreSequence,LastExonSequence
0,Lypla1,chr1,+,4807822,4846739,4807913,4845016,TGA,2,gagcgcgcgcccggaggggcgggcTGGGACTTTCGGCTGCCGGGAG...,38917,4844962,4846739,38917,1777,37140,54,1723,gagcgcgcgcccggaggggcgggcTGGGACTTTCGGCTGCCGGGAG...,GAAATGATGGATGTCAAGCACTTCATTGATAAGCTCCTACCTCCAA...
1,Gm37988,chr1,+,4807891,4886770,4807913,4832316,TGA,3,CGCACTGTCCGCCAGCCGGTGGATGTGCGGCAACAACATGTCCGCT...,78879,4886743,4886770,78879,27,78852,-54427,54454,CGCACTGTCCGCCAGCCGGTGGATGTGCGGCAACAACATGTCCGCT...,ATGGACCATCAACTGATAAAGACCCTG
2,Tcea1,chr1,+,4857813,4897905,4857913,4896364,TGA,2,GGAAGGCCTAGGCCAGCGGCTTCGCGGCTTGTCCAACGTCCGCGCA...,40092,4896355,4897905,40092,1550,38542,9,1541,GGAAGGCCTAGGCCAGCGGCTTCGCGGCTTGTCCAACGTCCGCGCA...,TTCTGTTGAGTTTGGAAGAATTGGCAAAGTATCTGGACCATTAAGA...
3,Atp6v1h,chr1,+,5083079,5162529,5084450,5162165,TGA,2,AGCTGCGGTCACGTGCGGCTGCTCACGTGCCAGACGCTGCTGTCAC...,79450,5162104,5162529,79450,425,79025,61,364,AGCTGCGGTCACGTGCGGCTGCTCACGTGCCAGACGCTGCTGTCAC...,GGAATATCTTGGTAAACAACTACAGTCAGAGCAGCCTCAGACTGCT...
4,Oprk1,chr1,+,5588492,5606131,5589048,5602784,TGA,2,GCGCACCTTGCTGATCCCAAACAGGCAGAGCTTCTTCCAGTCTTGG...,17639,5602251,5606131,17639,3880,13759,533,3347,GCGCACCTTGCTGATCCCAAACAGGCAGAGCTTCTTCCAGTCTTGG...,ATGTGGATGTCATTGAATGCTCCTTGCAGTTTCCTGATGATGAATA...
5,Rb1cc1,chr1,+,6214644,6276648,6230002,6274275,TAG,2,GCTCTGCGGCGGAAAGGCCGAGTCGACAATAACAAACCCCACGGCG...,62004,6274197,6276648,62004,2451,59553,78,2373,GCTCTGCGGCGGAAAGGCCGAGTCGACAATAACAAACCCCACGGCG...,GCACAAAACAGATTTAAAGTTCCTTTGGGGACAAAGTTTTACAGAG...
6,Fam150a,chr1,+,6359217,6394731,6359330,6391104,TAG,3,CGCCCCGCCTCCGGTCCCCGCCCAGCGCCCTAGAGCGTTTAGACCT...,35514,6394269,6394731,35514,462,35052,-3165,3627,CGCCCCGCCTCCGGTCCCCGCCCAGCGCCCTAGAGCGTTTAGACCT...,ACATTTCATAAGAAGCAAATATAATTTGAAGAAAAAGTGCCTGGAA...
7,St18,chr1,+,6487230,6860934,6768897,6858600,TAG,2,GAAAAGAGTTTTGAAATGAGTTCTGAATAGACAGCATTTTCTTTGT...,373704,6858459,6860934,373704,2475,371229,141,2334,GAAAAGAGTTTTGAAATGAGTTCTGAATAGACAGCATTTTCTTTGT...,GGGCCTATCAATGAGCAGAATTTTGAAGCATATGTAAATACTCTCA...
8,Pcmtd1,chr1,+,7088919,7173628,7120308,7169882,TAA,2,GGCATCTTTCTTCGGAGGGCGCCCCGCGCACGCGCAGCTCCCAGGA...,84709,7169514,7173628,84709,4114,80595,368,3746,GGCATCTTTCTTCGGAGGGCGCCCCGCGCACGCGCAGCTCCCAGGA...,CTCCATGTGCTGTCAGGAATCTCCAAGACTTGGCTCGCATTTACAT...
9,Rrs1,chr1,+,9545407,9547455,9545523,9546621,TAG,1,TCTTCCGGATTACGGGTCCGGACGTCCGCACGTGGTTGCCGGTTTA...,2048,9545407,9547455,2048,2048,0,1214,834,,TCTTCCGGATTACGGGTCCGGACGTCCGCACGTGGTTGCCGGTTTA...


In [25]:
Fasta['StopCodonSequence'] = 0

In [26]:
for i in range(len(Fasta)):
    Fasta.loc[i,("StopCodonSequence")] = Fasta.loc[i,("LastExonSequence")][(Fasta.loc[i,("CodingLength")]-3):(Fasta.loc[i,("CodingLength")])]

In [27]:
Fasta

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,StopCodon,Group,Sequence,...,LastExonStart,LastExonEnd,GeneLength,LastExonLength,Pre,CodingLength,NoncodingLength,PreSequence,LastExonSequence,StopCodonSequence
0,Lypla1,chr1,+,4807822,4846739,4807913,4845016,TGA,2,gagcgcgcgcccggaggggcgggcTGGGACTTTCGGCTGCCGGGAG...,...,4844962,4846739,38917,1777,37140,54,1723,gagcgcgcgcccggaggggcgggcTGGGACTTTCGGCTGCCGGGAG...,GAAATGATGGATGTCAAGCACTTCATTGATAAGCTCCTACCTCCAA...,TGA
1,Gm37988,chr1,+,4807891,4886770,4807913,4832316,TGA,3,CGCACTGTCCGCCAGCCGGTGGATGTGCGGCAACAACATGTCCGCT...,...,4886743,4886770,78879,27,78852,-54427,54454,CGCACTGTCCGCCAGCCGGTGGATGTGCGGCAACAACATGTCCGCT...,ATGGACCATCAACTGATAAAGACCCTG,
2,Tcea1,chr1,+,4857813,4897905,4857913,4896364,TGA,2,GGAAGGCCTAGGCCAGCGGCTTCGCGGCTTGTCCAACGTCCGCGCA...,...,4896355,4897905,40092,1550,38542,9,1541,GGAAGGCCTAGGCCAGCGGCTTCGCGGCTTGTCCAACGTCCGCGCA...,TTCTGTTGAGTTTGGAAGAATTGGCAAAGTATCTGGACCATTAAGA...,TGA
3,Atp6v1h,chr1,+,5083079,5162529,5084450,5162165,TGA,2,AGCTGCGGTCACGTGCGGCTGCTCACGTGCCAGACGCTGCTGTCAC...,...,5162104,5162529,79450,425,79025,61,364,AGCTGCGGTCACGTGCGGCTGCTCACGTGCCAGACGCTGCTGTCAC...,GGAATATCTTGGTAAACAACTACAGTCAGAGCAGCCTCAGACTGCT...,TGA
4,Oprk1,chr1,+,5588492,5606131,5589048,5602784,TGA,2,GCGCACCTTGCTGATCCCAAACAGGCAGAGCTTCTTCCAGTCTTGG...,...,5602251,5606131,17639,3880,13759,533,3347,GCGCACCTTGCTGATCCCAAACAGGCAGAGCTTCTTCCAGTCTTGG...,ATGTGGATGTCATTGAATGCTCCTTGCAGTTTCCTGATGATGAATA...,TGA
5,Rb1cc1,chr1,+,6214644,6276648,6230002,6274275,TAG,2,GCTCTGCGGCGGAAAGGCCGAGTCGACAATAACAAACCCCACGGCG...,...,6274197,6276648,62004,2451,59553,78,2373,GCTCTGCGGCGGAAAGGCCGAGTCGACAATAACAAACCCCACGGCG...,GCACAAAACAGATTTAAAGTTCCTTTGGGGACAAAGTTTTACAGAG...,TAG
6,Fam150a,chr1,+,6359217,6394731,6359330,6391104,TAG,3,CGCCCCGCCTCCGGTCCCCGCCCAGCGCCCTAGAGCGTTTAGACCT...,...,6394269,6394731,35514,462,35052,-3165,3627,CGCCCCGCCTCCGGTCCCCGCCCAGCGCCCTAGAGCGTTTAGACCT...,ACATTTCATAAGAAGCAAATATAATTTGAAGAAAAAGTGCCTGGAA...,
7,St18,chr1,+,6487230,6860934,6768897,6858600,TAG,2,GAAAAGAGTTTTGAAATGAGTTCTGAATAGACAGCATTTTCTTTGT...,...,6858459,6860934,373704,2475,371229,141,2334,GAAAAGAGTTTTGAAATGAGTTCTGAATAGACAGCATTTTCTTTGT...,GGGCCTATCAATGAGCAGAATTTTGAAGCATATGTAAATACTCTCA...,TAG
8,Pcmtd1,chr1,+,7088919,7173628,7120308,7169882,TAA,2,GGCATCTTTCTTCGGAGGGCGCCCCGCGCACGCGCAGCTCCCAGGA...,...,7169514,7173628,84709,4114,80595,368,3746,GGCATCTTTCTTCGGAGGGCGCCCCGCGCACGCGCAGCTCCCAGGA...,CTCCATGTGCTGTCAGGAATCTCCAAGACTTGGCTCGCATTTACAT...,TAA
9,Rrs1,chr1,+,9545407,9547455,9545523,9546621,TAG,1,TCTTCCGGATTACGGGTCCGGACGTCCGCACGTGGTTGCCGGTTTA...,...,9545407,9547455,2048,2048,0,1214,834,,TCTTCCGGATTACGGGTCCGGACGTCCGCACGTGGTTGCCGGTTTA...,TAG


In [28]:
Fasta = Fasta[Fasta["Group"] == 2]
Fasta.reset_index(drop=True, inplace=True)
Fasta = Fasta[Fasta["StopCodon"] == "TGA"]
Fasta.reset_index(drop=True, inplace=True)

In [29]:
Fasta

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,StopCodon,Group,Sequence,...,LastExonStart,LastExonEnd,GeneLength,LastExonLength,Pre,CodingLength,NoncodingLength,PreSequence,LastExonSequence,StopCodonSequence
0,Lypla1,chr1,+,4807822,4846739,4807913,4845016,TGA,2,gagcgcgcgcccggaggggcgggcTGGGACTTTCGGCTGCCGGGAG...,...,4844962,4846739,38917,1777,37140,54,1723,gagcgcgcgcccggaggggcgggcTGGGACTTTCGGCTGCCGGGAG...,GAAATGATGGATGTCAAGCACTTCATTGATAAGCTCCTACCTCCAA...,TGA
1,Tcea1,chr1,+,4857813,4897905,4857913,4896364,TGA,2,GGAAGGCCTAGGCCAGCGGCTTCGCGGCTTGTCCAACGTCCGCGCA...,...,4896355,4897905,40092,1550,38542,9,1541,GGAAGGCCTAGGCCAGCGGCTTCGCGGCTTGTCCAACGTCCGCGCA...,TTCTGTTGAGTTTGGAAGAATTGGCAAAGTATCTGGACCATTAAGA...,TGA
2,Atp6v1h,chr1,+,5083079,5162529,5084450,5162165,TGA,2,AGCTGCGGTCACGTGCGGCTGCTCACGTGCCAGACGCTGCTGTCAC...,...,5162104,5162529,79450,425,79025,61,364,AGCTGCGGTCACGTGCGGCTGCTCACGTGCCAGACGCTGCTGTCAC...,GGAATATCTTGGTAAACAACTACAGTCAGAGCAGCCTCAGACTGCT...,TGA
3,Oprk1,chr1,+,5588492,5606131,5589048,5602784,TGA,2,GCGCACCTTGCTGATCCCAAACAGGCAGAGCTTCTTCCAGTCTTGG...,...,5602251,5606131,17639,3880,13759,533,3347,GCGCACCTTGCTGATCCCAAACAGGCAGAGCTTCTTCCAGTCTTGG...,ATGTGGATGTCATTGAATGCTCCTTGCAGTTTCCTGATGATGAATA...,TGA
4,Sgk3,chr1,+,9798106,9900843,9865252,9898890,TGA,2,gcccgccgcccgGCGCTTCGCGGGAGGAGCCGCCGTGACGCGTCAG...,...,9898719,9900843,102737,2124,100613,171,1953,gcccgccgcccgGCGCTTCGCGGGAGGAGCCGCCGTGACGCGTCAG...,GCTGGACCAGATGATATCAGAAACTTTGATGCCGTCTTCACTGAAG...,TGA
5,Prex2,chr1,+,10993464,11303681,10993792,11297753,TGA,2,ATTTGATTGATGTCCAAGTCACCGCTGTGGCTGGAGCCGCCTCCCC...,...,11297707,11303681,310217,5974,304243,46,5928,ATTTGATTGATGTCCAAGTCACCGCTGTGGCTGGAGCCGCCTCCCC...,GCTGTACAAGCTCTGCGAGCCACCACCTCCAGTAGGAGAAGAATGA...,TGA
6,Kcnb2,chr1,+,15312451,15723750,15312451,15711629,TGA,2,ATGGCAGAAAAGGCACCACCTGGCTTGAACAGAAAGACATCCAGGT...,...,15709484,15723750,411299,14266,397033,2145,12121,ATGGCAGAAAAGGCACCACCTGGCTTGAACAGAAAGACATCCAGGT...,ATCCTGGCCATCGTGTCTATCCTGTTCATCGTTCTTTCCACCATTG...,TGA
7,Terf1,chr1,+,15805645,15844052,15805677,15843082,TGA,2,AGCTGAGGCACGGCGAGCGCTTTCGGTTTAACATGGCGGAGACGGT...,...,15842920,15844052,38407,1132,37275,162,970,AGCTGAGGCACGGCGAGCGCTTTCGGTTTAACATGGCGGAGACGGT...,ACATGGCTTTGGGAAGAAGACAGAATTTTGAAGTGTGGTGTAAAGA...,TGA
8,Tmem70,chr1,+,16665209,16678275,16665307,16677425,TGA,2,GCCTCATCGCTCTTGTGGGGGCCACTGTCGAGCGGAGTCGCTTTTG...,...,16676967,16678275,13066,1308,11758,458,850,GCCTCATCGCTCTTGTGGGGGCCACTGTCGAGCGGAGTCGCTTTTG...,GTGTGAAATGTTTCTCTTATTCTACAAGTGTTGTCAGCCTTGCATT...,TGA
9,Crispld1,chr1,+,17727087,17766344,17728656,17764114,TGA,2,TGCCCACTTACCTCAAGTTCTGGGGCTAGCTCCTTTGCAACCAGAC...,...,17764062,17766344,39257,2282,36975,52,2230,TGCCCACTTACCTCAAGTTCTGGGGCTAGCTCCTTTGCAACCAGAC...,TTTACAGAACCCTACAGGAGGAAAGGCATTCCGTGTATTCGCTGTT...,TGA


### Select positive strand

In [30]:
Fasta_Pos = Fasta[Fasta["strand"]=="+"]
Fasta_Pos = Fasta_Pos.reset_index(drop = True)

In [31]:
Fasta_Pos["Stop"] = Fasta_Pos["cdsEnd"] - Fasta_Pos["txStart"]

In [32]:
Sequence = Fasta_Pos["Sequence"].tolist()
Stop = Fasta_Pos["Stop"].tolist()
Pre = Fasta_Pos["Pre"].tolist()
CodingLength = Fasta_Pos["CodingLength"].tolist()
NoncodingLength = Fasta_Pos["NoncodingLength"].tolist()

In [33]:
for i in range(len(Sequence)):
    input_sequence = Sequence[i]
    input_sequence = input_sequence.upper()
    seq = input_sequence[(Stop[i]-3):(Stop[i])]
    print(seq)

TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA


TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA


In [34]:
df = pd.DataFrame({"Index":range(0, 2005)})
for i in range(len(Sequence)):
    input_sequence = Sequence[i]
    seq = input_sequence[:(Stop[i]-3)] + "TAG" + input_sequence[(Stop[i]):]
    x = one_hot_encode('N'*(context//2) + seq + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    m6AAI_prob = y[0, :, 1]
    
    m6AAI_prob = m6AAI_prob.tolist()
    m6AAI_prob = m6AAI_prob[Pre[i]:]
    
        
    if CodingLength[i] >= 1004:
        m6AAI_prob = m6AAI_prob[(CodingLength[i]-3-1000-1):(CodingLength[i]+1002-1)]
    if CodingLength[i] < 1004:
        m6AAI_prob = [0]*(1004-CodingLength[i]) + m6AAI_prob
    
    if len(m6AAI_prob) < 2005:
        m6AAI_prob = m6AAI_prob + [0]*(2005-len(m6AAI_prob))
    
    New = np.array(m6AAI_prob)
    New = pd.DataFrame({Fasta_Pos.loc[i,"name"]:New})
    df = pd.merge(df, New, left_index=True, right_index=True)    

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':


### Select negative strand

In [35]:
Fasta_Neg = Fasta[Fasta["strand"]=="-"]
Fasta_Neg = Fasta_Neg.reset_index(drop = True)

In [36]:
Fasta_Neg["Stop"] = Fasta_Neg["txEnd"] - Fasta_Neg["cdsStart"]

In [37]:
Sequence = Fasta_Neg["Sequence"].tolist()
Stop = Fasta_Neg["Stop"].tolist()
Pre = Fasta_Neg["Pre"].tolist()
CodingLength = Fasta_Neg["CodingLength"].tolist()
NoncodingLength = Fasta_Neg["NoncodingLength"].tolist()

In [38]:
for i in range(len(Sequence)):
    input_sequence = Sequence[i]
    input_sequence = input_sequence.upper()
    seq = input_sequence[(Stop[i]-3):(Stop[i])]
    print(seq)

TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA


TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA
TGA


In [39]:
for i in range(len(Sequence)):
    input_sequence = Sequence[i]
    seq = input_sequence[:(Stop[i]-3)] + "TAG" + input_sequence[(Stop[i]):]
    x = one_hot_encode('N'*(context//2) + seq + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    m6AAI_prob = y[0, :, 1]
    
    m6AAI_prob = m6AAI_prob.tolist()
    m6AAI_prob = m6AAI_prob[Pre[i]:]
    
        
    if CodingLength[i] >= 1004:
        m6AAI_prob = m6AAI_prob[(CodingLength[i]-3-1000-1):(CodingLength[i]+1002-1)]
    if CodingLength[i] < 1004:
        m6AAI_prob = [0]*(1004-CodingLength[i]) + m6AAI_prob
    
    if len(m6AAI_prob) < 2005:
        m6AAI_prob = m6AAI_prob + [0]*(2005-len(m6AAI_prob))
    
    New = np.array(m6AAI_prob)
    New = pd.DataFrame({Fasta_Neg.loc[i,"name"]:New})
    df = pd.merge(df, New, left_index=True, right_index=True)    


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':


In [40]:
df.drop(["Index"], axis=1, inplace=True)
df = df.T

In [41]:
df.loc["col_sum"] = df.sum()
df = df.T

In [42]:
Value = df[["col_sum"]]

In [43]:
Value

,col_sum
0,1.491909
1,0.563926
2,3.082538
3,1.435500
4,1.635311
5,3.312322
6,1.732710
7,0.974734
8,1.420658
9,1.297810


In [44]:
Value.to_csv("Probability_TGA2TAG.csv")

In [45]:
df.drop(["col_sum"], axis=1, inplace=True)
df = df.T

In [46]:
df.loc["Number"] = (df > 0).sum()
df = df.T

In [47]:
Number = df[["Number"]]

In [48]:
Number 

,Number
0,567.0
1,570.0
2,570.0
3,571.0
4,571.0
5,572.0
6,572.0
7,574.0
8,575.0
9,575.0


In [49]:
Number.to_csv("Number_TGA2TAG.csv")